# SimpliVity PowerShell REST API Example

This sample of Windows PowerShell code performs authentication to the SimpliVity REST API endpoint which is either an OVC(Omnistack-Virtual-Controller) or the MVA(Management-Virtual-Appliance) to the designated federation. <br>
You will be able to use the assigned username and password to perform a demo of this snippet of code. If you take a look at the Introduction notebook, opened along with this JupyterHub notebook, then you can see the datacenter that <br>
you are working on. For today's exercise, you will be using one of the IP addresses from the OVC in the 1st datacenter, and 1st Cluster.

> Note that the example in this Lab is based on **Powershell Core v7.0** running on Linux 


## SimpliVity Backup list and Rename
Once the access token is acquired, you should be able to obtain the SimpliVity backup of backup list and perform
the rename of the SimpliVity backup. 

- **Note** The access token will expire after 10 minutes of no activity or after 24 hours with activity.

- **Note** The POST method in SimpliVity is asynchronous, hence the user must use the Task ID returned to check on completion as shown in changing the backup name as shown in the example below
![IP-addreses](Pictures/HPEDEV-OVC-IP-Addresses.gif)
### Let's load up the credential (root/vmware) and the IP address of the OVC(16.31.87.101) <nt>

In [1]:
# Base URL for the SimpliVity authentication
$Base_URL = "https://16.31.87.101/api/"

# Set the usernamd and password
# change the X to the number corresponding to your assigned username studentXX"
$hms_username = 'root'
$hms_password = 'vmware'

# Allow the use of self-signed certificates and configure the SSL tunnel with TLS 1.2
[System.Net.ServicePointManager]::ServerCertificateValidationCallback = { $True }
[System.Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12

### Authenticate user and create structure required to generate access token

In [ ]:
# Create a base64 encoding for HTTP Authentication.
$base64AuthInfo = [Convert]::ToBase64String([Text.Encoding]::ASCII.GetBytes(("{0}:{1}" -f "simplivity","")))
Write-Output "Base64AuthInfo= $base64AuthInfo `n" 

# Create a JSON body with username, password, and grant_type.
$body = @{grant_type='password';username=$hms_username;password=$hms_password}
Write-Output "REST Body=", $body, "`n"

# Authenticate the user and create the structure to generate an access token.
$url = $BASE_URL+'oauth/token'
Write-Output "URL= $url`n"
$header = @{Authorization=("Basic {0}" -f $base64AuthInfo)}
Write-Output "REST Header=", $header, "`n"

In [ ]:
# Invoke the REST API and
$response = Invoke-RestMethod -SkipCertificateCheck -Uri $url -Headers $header -Body $body -Method Post
$response

In [ ]:
# Obtain the access token
$access_token = $response.access_token

### Use the token to get the list of the backups inside the LasVegas Fed

In [ ]:
# Add the access_token to the header.
$header = @{Authorization='Bearer '+$access_token}

In [ ]:
# Issue a GET request: GET /backups.
$url = $BASE_URL+'backups'
$backups = Invoke-RestMethod -Header $header -Uri $url -SkipCertificateCheck

In [ ]:
$backups | select -expandproperty backups | select Name,virtual_machine_name,type,datastore_name -first 40 | format-table

In [ ]:
# Obtain the backup ID of the nth backup from the list where nth is random no from 1 to 40
$count = get-random -Maximum 40 -Minimum 1
$backupid = $backups.backups[$count].id
$backupName = $backups.backups[$count].Name
write-output ($count, $backupid, $backupName)

### Use the token to change the name of the nth backup to today's date inside the LasVegas Fed

In [ ]:
# Create a JSON body for the rename action. The name is combination of the hms_user_name and the date
$date = get-date 
$body = @{backup_name=("{0}: {1}" -f "student$$I", $date)}
$body = $body | ConvertTo-Json
$body

In [ ]:
# Form the URI for the changing the name
$url = $BASE_URL+'backups/'
$url += $backupid
$url += '/rename'

In [ ]:
# Issue the POST operation and expect a task object in return.

$response = Invoke-RestMethod -SkipCertificateCheck -Header $header -Uri $url -Method Post -Body $body -ContentType 'application/vnd.simplivity.v1+json'
$response.task.id

In [ ]:
$response.task

In [ ]:
# Monitor the status of the rename operation by using a loop to query the task while this task is IN_PROGRESS.
# The state field in the JSON response body indicates the status.

$taskid = $response.task.id
$state = $response.task.state
$url = $BASE_URL+'tasks/'+$taskid
  while ($state -eq 'IN_PROGRESS') {
    # Wait one second and try again.
    Start-Sleep -s 0.1
    $response = Invoke-RestMethod -SkipCertificateCheck -Header $header -Uri $url
    # Print out the task result.
    $state = $response.task.state
    $state
  }

### Let's take a look at the list of the backups of the changed names

In [ ]:

# Issue a GET request: GET /backups.
$url = $BASE_URL+'backups'
$backups = Invoke-RestMethod -Header $header -Uri $url -SkipCertificateCheck
$backups | select -expandproperty backups | select Name,virtual_machine_name,type,datastore_name -first 40 | format-table

### Congratulation, you have completed Introduction to SimpliVity REST API via Powershell

If you go to the lasvegasvc.demo.local, search the backups in the LasVegas Cluster with backup name to match Hacker to confirm that you now can interact with LasVegas SimpliVity fed using REST API and PowerShell


### Simplivity Object models and the relationships

More information on SimpliVity objects and resources are available at the https://developer.hpe.com/api/simplivity/

Here is the basic diagram of the Simplivity object models

![Simplivity Object](https://developer.hpe.com/uploads/media/2018/7/svt-rest-api-object-type-relationships-1532712788492.png)

# Continue now to Lab 2: PowerShell Module Example
Description: Let us use both Simplivity and VMware modules to perform cloning and power-on a designated VM.

* [Lab 2](PowerShellModuleExample.ipynb)


# Or Move back to the Introduction Notebook
Description: This will allow you to try other notebooks.

* [Introduction](Introduction.ipynb)